In [19]:
import os
import numpy as np
import cv2
import csv
from glob import glob
import matplotlib.pyplot as plt
from collections import namedtuple
from copy import deepcopy
from tqdm import tqdm
import random
import torch

In [20]:
# TODO
src = 'data/image-matching-challenge-2022/train'

val_scenes = []
for f in os.scandir(src):
    if f.is_dir():
        cur_scene = os.path.split(f)[-1]
        print(f'Found scene "{cur_scene}"" at {f.path}')
        val_scenes += [cur_scene]

selected_scenes = ["brandenburg_gate","grand_place_brussels","sagrada_familia"]

# TODO
scene = "brandenburg_gate"

images_dict = {}
for filename in glob(f'{src}/{scene}/images/*.jpg'):
    cur_id = os.path.basename(os.path.splitext(filename)[0])
    # OpenCV expects BGR, but the images are encoded in standard RGB
    images_dict[cur_id] = cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2RGB)
    
print(f'Loaded {len(images_dict)} images.')

def read_covisibility_data(filepath):
    """Read covisibility data from a CSV file and return a dictionary."""
    covisibility = {}
    with open(filepath, mode='r') as file:
        reader = csv.reader(file, delimiter=',')
        next(reader)  # Skip the header
        covisibility = {row[0]: float(row[1]) for row in reader}
    return covisibility

# Load covisibility data
covisibility_dict = read_covisibility_data(f'{src}/{scene}/pair_covisibility.csv')

pairs = [key for key, value in covisibility_dict.items()]
# Subset classification based on covisibility scores
easy_subset = [key for key, value in covisibility_dict.items() if value >= 0.7]
difficult_subset = [key for key, value in covisibility_dict.items() if 0.1 <= value < 0.2]
bad_subset = [key for key, value in covisibility_dict.items() if value < 0.1]


Found scene "british_museum"" at data/image-matching-challenge-2022/train/british_museum
Found scene "sacre_coeur"" at data/image-matching-challenge-2022/train/sacre_coeur
Found scene "brandenburg_gate"" at data/image-matching-challenge-2022/train/brandenburg_gate
Found scene "piazza_san_marco"" at data/image-matching-challenge-2022/train/piazza_san_marco
Found scene "buckingham_palace"" at data/image-matching-challenge-2022/train/buckingham_palace
Found scene "colosseum_exterior"" at data/image-matching-challenge-2022/train/colosseum_exterior
Found scene "pantheon_exterior"" at data/image-matching-challenge-2022/train/pantheon_exterior
Found scene "st_peters_square"" at data/image-matching-challenge-2022/train/st_peters_square
Found scene "sagrada_familia"" at data/image-matching-challenge-2022/train/sagrada_familia
Found scene "notre_dame_front_facade"" at data/image-matching-challenge-2022/train/notre_dame_front_facade
Found scene "st_pauls_cathedral"" at data/image-matching-challen

# Evaluation with fundamental matrix

In [6]:
# Load ground truth

# Named tuple to store camera intrinsics and extrinsics
CameraCalibration = namedtuple('CameraCalibration', ['K', 'R', 'T'])

def load_calibration(filepath):
    """Load calibration data from a CSV file."""
    calibration_data = {}
    with open(filepath, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        for row in reader:
            camera_id = row[0]
            K = np.array(list(map(float, row[1].split()))).reshape(3, 3)
            R = np.array(list(map(float, row[2].split()))).reshape(3, 3)
            T = np.array(list(map(float, row[3].split())))
            calibration_data[camera_id] = CameraCalibration(K=K, R=R, T=T)
    return calibration_data

def load_scaling_factors(filepath):
    """Load scaling factors from a CSV file."""
    scaling_factors = {}
    with open(filepath, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        for row in reader:
            scaling_factors[row[0]] = float(row[1])
    return scaling_factors

# Load calibration data and scaling factors
calib_dict = load_calibration(f'{src}/{scene}/calibration.csv')
print(f'Loaded ground truth data for {len(calib_dict)} images.\n')

scaling_dict = load_scaling_factors(f'{src}/scaling_factors.csv')
print(f'Scaling factors: {scaling_dict}\n')

Loaded ground truth data for 350 images.

Scaling factors: {'british_museum': 2.517, 'brandenburg_gate': 7.38, 'buckingham_palace': 18.75, 'colosseum_exterior': 36.99, 'grand_place_brussels': 10.26, 'lincoln_memorial_statue': 1.85, 'notre_dame_front_facade': 1.36, 'pantheon_exterior': 5.41, 'piazza_san_marco': 7.92, 'sacre_coeur': 20.27, 'sagrada_familia': 4.2, 'st_pauls_cathedral': 7.01, 'st_peters_square': 21.48, 'taj_mahal': 20.76, 'temple_nara_japan': 7.79, 'trevi_fountain': 3.67}



In [ ]:
def normalize_keypoints(keypoints, K):
    """Normalize keypoints using the camera intrinsics."""
    cx, cy = K[0, 2], K[1, 2]
    fx, fy = K[0, 0], K[1, 1]
    return (keypoints - np.array([[cx, cy]])) / np.array([[fx, fy]])

def compute_essential_matrix(F, K1, K2, kp1, kp2):
    """Compute the Essential matrix from the Fundamental matrix and keypoints."""
    E = K2.T @ F @ K1
    kp1_normalized = normalize_keypoints(kp1, K1)
    kp2_normalized = normalize_keypoints(kp2, K2)
    _, R, T, _ = cv2.recoverPose(E, kp1_normalized, kp2_normalized)
    return E, R, T


def quaternion_from_matrix(matrix):
    """Convert a rotation matrix into a quaternion."""
    M = np.array(matrix, dtype=np.float64, copy=False)[:3, :3]
    trace = np.trace(M)
    if trace > 0:
        s = np.sqrt(trace + 1.0) * 2
        qw = 0.25 * s
        qx = (M[2, 1] - M[1, 2]) / s
        qy = (M[0, 2] - M[2, 0]) / s
        qz = (M[1, 0] - M[0, 1]) / s
    else:
        idx = np.argmax(np.diagonal(M))
        if idx == 0:
            s = np.sqrt(1.0 + M[0, 0] - M[1, 1] - M[2, 2]) * 2
            qw = (M[2, 1] - M[1, 2]) / s
            qx = 0.25 * s
            qy = (M[0, 1] + M[1, 0]) / s
            qz = (M[0, 2] + M[2, 0]) / s
        elif idx == 1:
            s = np.sqrt(1.0 + M[1, 1] - M[0, 0] - M[2, 2]) * 2
            qw = (M[0, 2] - M[2, 0]) / s
            qx = (M[0, 1] + M[1, 0]) / s
            qy = 0.25 * s
            qz = (M[1, 2] + M[2, 1]) / s
        else:
            s = np.sqrt(1.0 + M[2, 2] - M[0, 0] - M[1, 1]) * 2
            qw = (M[1, 0] - M[0, 1]) / s
            qx = (M[0, 2] + M[2, 0]) / s
            qy = (M[1, 2] + M[2, 1]) / s
            qz = 0.25 * s
    return np.array([qw, qx, qy, qz])


def compute_error(q_gt, T_gt, q, T, scale):
    """Compute the error metrics for rotation and translation."""
    eps = 1e-15
    q_gt_normalized = q_gt / (np.linalg.norm(q_gt) + eps)
    q_normalized = q / (np.linalg.norm(q) + eps)
    loss_q = max(eps, 1.0 - np.sum(q_gt_normalized * q_normalized)**2)
    err_q = np.arccos(1 - 2 * loss_q)

    T_gt_scaled = T_gt * scale
    T_scaled = T * np.linalg.norm(T_gt) * scale / (np.linalg.norm(T) + eps)
    err_t = min(np.linalg.norm(T_gt_scaled - T_scaled), np.linalg.norm(T_gt_scaled + T_scaled))

    return np.degrees(err_q), err_t

# OpenCV gives us the Fundamental matrix after RANSAC, and a mask over the input matches. The solution is clearly much cleaner, even though it may still contain outliers.
def evaluate_one_pair_from_torch(matched_kpts1_coords, matched_kpts2_coords, pair_id):
    image_id1, image_id2 = pair_id.split('-')

    F, inlier_mask = cv2.findFundamentalMat(
        matched_kpts1_coords, 
        matched_kpts2_coords, 
        method=cv2.USAC_MAGSAC, 
        ransacReprojThreshold=0.25, 
        confidence=0.99999, 
        maxIters=10000)

    inlier_kp_1 = matched_kpts1_coords[inlier_mask.ravel() == 1]
    inlier_kp_2 = matched_kpts1_coords[inlier_mask.ravel() == 1]

    # Compute the Essential matrix, rotation, and translation
    E, R, T = compute_essential_matrix(F, calib_dict[image_id1].K, calib_dict[image_id2].K, inlier_kp_1, inlier_kp_2)
    q = quaternion_from_matrix(R)
    T = T.flatten()

    # Compute ground truth pose differences
    R1_gt, T1_gt = calib_dict[image_id1].R, calib_dict[image_id1].T.reshape((3, 1))
    R2_gt, T2_gt = calib_dict[image_id2].R, calib_dict[image_id2].T.reshape((3, 1))
    dR_gt = R2_gt @ R1_gt.T
    dT_gt = (T2_gt - dR_gt @ T1_gt).flatten()
    q_gt = quaternion_from_matrix(dR_gt)

        # Compute errors
    err_q, err_t = compute_error(q_gt, dT_gt, q, T, scaling_dict[scene])
    return err_q, err_t

In [ ]:
def load_torch_file(file_path):
    try:
        loaded_torch = torch.load(file_path)
        
        kpts1 = loaded_torch['all_kpts0']
        kpts2 = loaded_torch['all_kpts1']
        matched_kpts1_coords = loaded_torch['matched_kpts0']
        matched_kpts2_coords = loaded_torch['matched_kpts1']
        inlier_kpts1 = loaded_torch['inlier_kpts0']
        inlier_kpts2 = loaded_torch['inlier_kpts1']
        print(f"Successfully loaded {file_path}")
        return kpts1, kpts2, matched_kpts1_coords, matched_kpts2_coords, inlier_kpts1, inlier_kpts2
    except Exception as e:
        print(f"Error loading .torch file: {e}")
        return None

#TODO change to pair ID
pair_id = pairs[0] # 90920828_5082887495-20133057_3035445116
file_path = 'output_1_result.torch'  # Replace with your .torch file path
kpts1, kpts2, matched_kpts1_coords, matched_kpts2_coords, inlier_kpts1, inlier_kpts2 = load_torch_file(file_path)
# print("kpts1", kpts1.shape, kpts1[0])
# print("matched_kpts1",matched_kpts1_coords.shape, matched_kpts1_coords[0])
# print("matched_kpts2",matched_kpts2_coords.shape, matched_kpts2_coords[0])
# print("inlier_kpts1",inlier_kpts1.shape, inlier_kpts1[0])

err_q, err_t = evaluate_one_pair_from_torch(matched_kpts1_coords, matched_kpts2_coords, pair_id)
print(f'Pair "{pair_id}", rotation error: {err_q:.2f}° | translation error: {err_t:.2f} m')

90920828_5082887495-20133057_3035445116
Successfully loaded output_1_result.torch
Pair "90920828_5082887495-20133057_3035445116", rotation error: 1.47° | translation error: 2.02 m


/tmp/ipykernel_8976/826225559.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_torch = torch.load(file_path)
